# Structuring COICOP 2018 information for classification

The COICOP 2018 classification contains unstructured information for its most detailed level of classification. This notebook leverages LLMs in order to structure this information in a format which can be used for similarity search.

In [1]:
import os
import time # Add delays because of free API rate limits
from datetime import datetime

import pandas as pd
from ftfy import fix_text
from unidecode import unidecode

from typing import List
from pydantic import BaseModel
from pydantic_ai import Agent
from pydantic_ai.models.mistral import MistralModel
from pydantic_ai.models.groq import GroqModel

from dotenv import dotenv_values

In [2]:
import nest_asyncio # Fix issues with Jupyter notebook event loop
nest_asyncio.apply()

Import environment variables with API keys

In [3]:
config = dotenv_values(".env")

Define the Pydantic model for parsing the additional information in each COICOP 2018 level 4

In [4]:
class CoicopDetails(BaseModel):
    examples: List[str]

Read COICOP 2018 definitions

In [5]:
usecols = ["code","title","intro", "includes", "alsoIncludes"]

data_df = pd.read_excel(
    "coicop_2018/COICOP_2018_English_structure.xlsx", 
    usecols=usecols,)


Remove exclusion note from the English and French files, rename Spanish columns.

Remove (ND), (SD), (D), (S) markings from the class names

Filter level 4 classes only for LLMs queries

In [6]:
def process_subclasses(df):
    """
    Process DataFrame with the following operations:
    1. Filter rows where Code contains exactly 3 dots
    2. Combine description columns
    3. Remove classification markers from Description
    
    Args:
        df (pd.DataFrame): Input DataFrame
    
    Returns:
        pd.DataFrame: Processed DataFrame
    """
    # Create a copy to avoid modifying the original
    result_df = df.copy()
    
    # 1. Select rows where Code has exactly 2 dots
    result_df = result_df[result_df['code'].str.count(r'\.') == 3]
    
    # 2. Combine description columns
    columns_to_concat = ["intro", "includes", "alsoIncludes"]
    
    def concatenate_non_nan_columns(row):
        # Filter out NaN values and convert to string
        non_nan_values = [str(row[col]) for col in columns_to_concat if pd.notna(row[col])]
        return ' \n '.join(non_nan_values) if non_nan_values else ''
    
    result_df['description'] = result_df.apply(concatenate_non_nan_columns, axis=1)
    # Fix encoding issues
    result_df['description'] = result_df['description'].apply(fix_text)
    result_df['description'] = result_df['description'].apply(unidecode)
    result_df['description'] = result_df['description'].str.replace("_x000D_", " ")
        
    # 3. Remove classification markers from Description
    markers_pattern = r'\s*\((ND|SD|S|D)\)'
    result_df['title'] = result_df['title'].str.replace(markers_pattern, '', regex=True)
    
    return result_df[["code", "title", "description"]]

In [7]:
data_df = process_subclasses(data_df)

In [8]:
data_dict = data_df.to_dict(orient="records")

## Mistral

Initialize PydanticAI agent to structure information

In [49]:
llm_model = "mistral-large-latest"

In [50]:
model = MistralModel(model_name=llm_model, api_key=config.get("MISTRAL_API_KEY"))
agent = Agent(
    model=model,
    retries=3,
    result_type=CoicopDetails,
    system_prompt=(
        'You are an expert data curator. You will receive a string of text '
        'with examples of items to be included into a certain classification. '
        'Your task is to transform this text into a list of self-explainig '
        'items descriptions, exploding the examples to the most granular level you can '
        'identify in the text. '
        'Each item description should contain all necessary information for classification '
        'as provided in the original text. Prefer complete descriptions rather than single words. '
        'If possible, avoid the use of ambiguous or generic terms such as `other` or `miscellaneous`. '
        'Your output should be in the same language as the input text. '
        ),
)

Run calls to the Agent to extract and format information

In [51]:
results = []
failed_calls = []
for i, item in enumerate(data_dict):
    # Print every 20 items to show progress
    if i % 20 == 0:
        print(f"Processing item {i+1} out of {len(data_dict)}")
    # Add switch to skip none items
    if item.get("description") is None:
        # No information to parse, just append existing item
        results.append(item)
        continue
    # Time delay to respect API rate limits
    time.sleep(3)
    try:
        prompt = "{}. {}".format(item.get("title"), item.get("description"))
        agent_result = agent.run_sync(prompt, model_settings={'temperature': 0.0})
        # Add to results all examples, including the original class name...list concatenation
        for ex in [item.get("title")] + agent_result.data.model_dump().get("examples"):
            # the new "Description" is inserted at the end, so it overwrites the original one
            results.append({**item, "title": ex})
    except Exception as e:
        failed_calls.append(item)
        print(f"Error processing item {i+1} out of {len(data_dict)}")
        print(item)
        print(e)
        continue

Processing item 1 out of 338
Processing item 21 out of 338
Processing item 41 out of 338
Processing item 61 out of 338
Processing item 81 out of 338
Processing item 101 out of 338
Processing item 121 out of 338
Processing item 141 out of 338
Processing item 161 out of 338
Processing item 181 out of 338
Processing item 201 out of 338
Processing item 221 out of 338
Processing item 241 out of 338
Processing item 261 out of 338
Processing item 281 out of 338
Processing item 301 out of 338
Processing item 321 out of 338


Save results and failed calls

In [52]:
results_df = pd.DataFrame(results)
results_df.drop(columns=["description"]).to_csv(
    "results/coicop2018_{}_{}.csv".format(
        llm_model,
        datetime.now().strftime("%Y-%m-%d_%H%M%S")),
    index=False)

In [53]:
failed_df = pd.DataFrame(failed_calls)
failed_df.to_csv(
    "results/failed_coicop2018_{}_{}.csv".format(
        llm_model,
        datetime.now().strftime("%Y-%m-%d_%H%M%S")),
    index=False)

## Llama 3 on Groq

In [9]:
llm_model = "llama3-70b-8192"

In [10]:
model = GroqModel(
    model_name=llm_model, 
    api_key=config.get("GROQ_API_KEY"))
agent = Agent(
    model=model,
    retries=3,
    result_type=CoicopDetails,
    system_prompt=(
        'You are an expert data curator. You will receive a string of text '
        'with examples of items to be included into a certain classification. '
        'Your task is to transform this text into a list of self-explainig '
        'items descriptions, exploding the examples to the most granular level you can '
        'identify in the text. '
        'Each item description should contain all necessary information for classification '
        'as provided in the original text. Prefer complete descriptions rather than single words. '
        'If possible, avoid the use of ambiguous or generic terms such as `other` or `miscellaneous`. '
        'Your output should be in the same language as the input text. '
        ),
)

In [11]:
results = []
failed_calls = []
for i, item in enumerate(data_dict):
    # Print every 20 items to show progress
    if i % 20 == 0:
        print(f"Processing item {i+1} out of {len(data_dict)}")
    # Add switch to skip none items
    if item.get("description") is None:
        # No information to parse, just append existing item
        results.append(item)
        continue
    # Time delay to respect API rate limits
    time.sleep(3)
    try:
        prompt = "{}. {}".format(item.get("title"), item.get("description"))
        agent_result = agent.run_sync(prompt, model_settings={'temperature': 0.0})
        # Add to results all examples, including the original class name...list concatenation
        for ex in [item.get("title")] + agent_result.data.model_dump().get("examples"):
            # the new "Description" is inserted at the end, so it overwrites the original one
            results.append({**item, "title": ex})
    except Exception as e:
        failed_calls.append(item)
        print(f"Error processing item {i+1} out of {len(data_dict)}")
        print(item)
        print(e)
        continue

Processing item 1 out of 338
Error processing item 7 out of 338
{'code': '01.1.2.1', 'title': 'Live land animals', 'description': 'Live land animals, both domestic and wild, for human consumption as food. \n * cattle (cows; calves; common ox, zebu or humped ox; watusi, gaur, gayal, banteng and yak) \n* buffaloes (Indian or water buffaloes; arnis, or Asiatic buffaloes; Celebes anoas, or pygmy buffaloes; African buffaloes, such as the dwarf buffalo and the large Caffrarian buffalo; American bisons, known simply as buffaloes; European bisons; and beefaloes (hybrid offspring of American bisons and domestic cattle) \n* pigs \n* goats, lambs and sheep \n* poultry (chickens, ducks, geese, turkeys and guinea fowl) \n* hares and rabbits \n* horses, donkeys and other equines \n* camels, dromedaries, alpaca, llamas, guanacos, vicunas and other camelids \n* ostriches, emus, rheas, pheasants, grouse, pigeons, quails and other birds \n* snakes and other reptiles \n* spiders, scorpions, insects and w

In [12]:
results_df = pd.DataFrame(results)
results_df.drop(columns=["description"]).to_csv(
    "results/coicop2018_{}_{}.csv".format(
        llm_model,
        datetime.now().strftime("%Y-%m-%d_%H%M%S")),
    index=False)

In [13]:
failed_df = pd.DataFrame(failed_calls)
failed_df.to_csv(
    "results/failed_coicop2018_{}_{}.csv".format(
        llm_model,
        datetime.now().strftime("%Y-%m-%d_%H%M%S")),
    index=False)

## Mixtral on Groq

In [14]:
llm_model = "mixtral-8x7b-32768"

In [15]:
model = GroqModel(
    model_name=llm_model, 
    api_key=config.get("GROQ_API_KEY"))
agent = Agent(
    model=model,
    retries=3,
    result_type=CoicopDetails,
    system_prompt=(
        'You are an expert data curator. You will receive a string of text '
        'with examples of items to be included into a certain classification. '
        'Your task is to transform this text into a list of self-explainig '
        'items descriptions, exploding the examples to the most granular level you can '
        'identify in the text. '
        'Each item description should contain all necessary information for classification '
        'as provided in the original text. Prefer complete descriptions rather than single words. '
        'If possible, avoid the use of ambiguous or generic terms such as `other` or `miscellaneous`. '
        'Your output should be in the same language as the input text. '
        ),
)

In [16]:
results = []
failed_calls = []
for i, item in enumerate(data_dict):
    # Print every 20 items to show progress
    if i % 20 == 0:
        print(f"Processing item {i+1} out of {len(data_dict)}")
    # Add switch to skip none items
    if item.get("description") is None:
        # No information to parse, just append existing item
        results.append(item)
        continue
    # Time delay to respect API rate limits
    time.sleep(3)
    try:
        prompt = "{}. {}".format(item.get("title"), item.get("description"))
        agent_result = agent.run_sync(prompt, model_settings={'temperature': 0.0})
        # Add to results all examples, including the original class name...list concatenation
        for ex in [item.get("title")] + agent_result.data.model_dump().get("examples"):
            # the new "Description" is inserted at the end, so it overwrites the original one
            results.append({**item, "title": ex})
    except Exception as e:
        failed_calls.append(item)
        print(f"Error processing item {i+1} out of {len(data_dict)}")
        print(item)
        print(e)
        continue

Processing item 1 out of 338
Error processing item 4 out of 338
{'code': '01.1.1.4', 'title': 'Breakfast cereals', 'description': '* cornflakes, oatmeal and oat flakes, muesli, granola and puffed cereals including puffed rice cakes \n* breakfast cereals with nuts or dried fruit'}
Error code: 400 - {'error': {'message': "Failed to call a function. Please adjust your prompt. See 'failed_generation' for more details.", 'type': 'invalid_request_error', 'code': 'tool_use_failed', 'failed_generation': '<tool-use>\n{\n\t"tool_call": {\n\t\t"id": "pending",\n\t\t"type": "function",\n\t\t"function": {\n\t\t\t"name": "final\\_result"\n\t\t},\n\t\t"parameters": {\n\t\t\t"examples": [\n\t\t\t\t"cornflakes",\n\t\t\t\t"oatmeal and oat flakes",\n\t\t\t\t"muesli",\n\t\t\t\t"granola",\n\t\t\t\t"puffed cereals including puffed rice cakes",\n\t\t\t\t"breakfast cereals with nuts",\n\t\t\t\t"breakfast cereals with dried fruit"\n\t\t\t]\n\t\t}\n\t}\n}\n</tool-use>'}}
Error processing item 5 out of 338
{'cod

In [17]:
results_df = pd.DataFrame(results)
results_df.drop(columns=["description"]).to_csv(
    "results/coicop2018_{}_{}.csv".format(
        llm_model,
        datetime.now().strftime("%Y-%m-%d_%H%M%S")),
    index=False)

In [18]:
failed_df = pd.DataFrame(failed_calls)
failed_df.to_csv(
    "results/failed_coicop2018_{}_{}.csv".format(
        llm_model,
        datetime.now().strftime("%Y-%m-%d_%H%M%S")),
    index=False)

# Combine results

In [19]:
result_files = [f for f in os.listdir("results/") if f.startswith("coicop2018") and f.endswith(".csv")]

In [20]:
results_list = []
for f in result_files:
    temp_df = pd.read_csv(os.path.join("results", f))
    results_list.append(temp_df)

results_df = pd.concat(results_list)

In [21]:
results_df["code"].nunique()

338

Normalize to lowercase and remove all "other" labels

In [22]:
# Lowercase
results_df["title"] = results_df["title"].str.lower()
# Remove duplicates
results_df = results_df.drop_duplicates(ignore_index=True)
# Remove items with "other" or "miscellaneous"
results_df = results_df[~results_df["title"].str.contains("other|miscellaneous")]

In [23]:
results_df["code"].nunique()

337

In [24]:
results_df.to_csv(
    "results/consolidated_coicop2018_20250210.csv", index=False)